In [1]:
#### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w3-p1/train.csv
/kaggle/input/2021-ai-w3-p1/test.csv
/kaggle/input/2021-ai-w3-p1/submit_sample.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
# For reproducibility
torch.manual_seed(1)

In [4]:
train_data=pd.read_csv("/kaggle/input/2021-ai-w3-p1/train.csv")
test_data=pd.read_csv("/kaggle/input/2021-ai-w3-p1/test.csv")
submit_data=pd.read_csv("/kaggle/input/2021-ai-w3-p1/submit_sample.csv")
print(train_data.head())
print(test_data.head())
print(submit_data.head())

       year  avgTemp  minTemp  maxTemp  rainFall  avgPrice
0  20100101     -4.9    -11.0      0.9       0.0      2123
1  20100102     -3.1     -5.5      5.5       0.8      2123
2  20100103     -2.9     -6.9      1.4       0.0      2123
3  20100104     -1.8     -5.1      2.2       5.9      2020
4  20100105     -5.2     -8.7     -1.8       0.7      2060
       year  avgTemp  minTemp  maxTemp  rainFall
0  20170107      4.0     -1.4      9.5       0.1
1  20170120     -1.7     -5.4      1.4       3.3
2  20170223      1.4     -2.0      5.0       0.0
3  20170227      3.6     -2.5     11.1       0.1
4  20170309      5.0     -1.6     11.3       0.0
   Id  Expected
0   0      1000
1   1      1000
2   2      1000
3   3      1000
4   4      1000


In [5]:
x_train=np.array([train_data['avgTemp'],train_data['minTemp'],train_data['maxTemp'],train_data['rainFall']])
x_train=x_train.T
x_train=torch.FloatTensor(x_train)
y_train=torch.FloatTensor(train_data['avgPrice'])

In [6]:
print(x_train.shape)
print(y_train.shape)

torch.Size([2557, 4])
torch.Size([2557])


In [7]:
#모델초기화
W=torch.zeros((4,1), requires_grad=True)
b=torch.zeros(1, requires_grad=True)

In [8]:
#optimizer설정
optimizer=optim.SGD([W,b], lr=0.0001)

nb_epochs=5000
for epoch in range(nb_epochs + 1):
    #H(x)
    hypothesis=x_train.matmul(W)+b
    #Cost
    cost=torch.mean((hypothesis - y_train) ** 2)
    #change H(x)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    #500번 마다 출력
    if epoch % 500 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(epoch,nb_epochs,hypothesis.squeeze().detach(),cost.item()))

Epoch    0/5000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 12013343.000000
Epoch  500/5000 hypothesis: tensor([2525.0376, 2532.8330, 1805.8557,  ..., 1492.0636, 2602.8203,
        2369.4519]) Cost: 2406400.750000
Epoch 1000/5000 hypothesis: tensor([2827.7119, 2791.4763, 2016.6981,  ..., 1653.4608, 2858.8821,
        2559.3027]) Cost: 2373814.250000
Epoch 1500/5000 hypothesis: tensor([2857.7812, 2829.7754, 2043.3334,  ..., 1677.0248, 2885.4524,
        2581.7173]) Cost: 2366631.250000
Epoch 2000/5000 hypothesis: tensor([2861.4504, 2846.5366, 2052.0850,  ..., 1687.1879, 2889.7815,
        2587.8818]) Cost: 2359781.750000
Epoch 2500/5000 hypothesis: tensor([2862.5725, 2861.0249, 2059.0676,  ..., 1696.0063, 2891.9480,
        2592.4421]) Cost: 2353029.000000
Epoch 3000/5000 hypothesis: tensor([2863.4565, 2875.1069, 2065.8408,  ..., 1704.6471, 2893.8958,
        2596.8167]) Cost: 2346369.750000
Epoch 3500/5000 hypothesis: tensor([2864.3184, 2888.9609, 2072.5513,  ..., 1713.220

In [9]:
print(W)
print(b)

tensor([[ -37.0251],
        [-203.0161],
        [ 263.8333],
        [  39.9994]], requires_grad=True)
tensor([214.8767], requires_grad=True)


In [10]:
x_test=torch.FloatTensor(np.array([test_data['avgTemp'],test_data['minTemp'],test_data['maxTemp'],test_data['rainFall']]))
x_test=x_test.T
y_test=x_test.matmul(W)+b
print(y_test)

tensor([[2861.4153],
        [1875.4711],
        [1888.2404],
        [3521.6765],
        [3335.8933],
        [3418.6145],
        [2967.6013],
        [4374.0698],
        [3649.7529],
        [4523.0508],
        [2118.9209],
        [3835.0054],
        [2943.0647],
        [3165.0242],
        [2453.0134],
        [3531.3711],
        [3459.7830],
        [4397.7646],
        [3141.9841],
        [3488.5959],
        [2485.1023],
        [2841.3582],
        [3220.1492],
        [1679.0319]], grad_fn=<AddBackward0>)


In [11]:
submit_data['Expected']=y_test.detach().numpy()
print(submit_data.head())

   Id     Expected
0   0  2861.415283
1   1  1875.471069
2   2  1888.240356
3   3  3521.676514
4   4  3335.893311


In [12]:
submit_data.to_csv("submission.csv",mode='w',index=False)